## No guardrails

A search application without guardrails. We want to apply Guardrails to this application without modifying the application code.

In [1]:
#%pip install --quiet llama-index llama-index-retrievers-bm25 llama-index-llms-anthropic anthropic

In [1]:
#GEMINI="gemini-2.0-flash"
#OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
#assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
#       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
#assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
#       "Please specify the OPENAI_API_KEY access token in keys.env file"

## Basic RAG application

This is the application that we want to protect.

In [2]:
from basic_rag import build_query_engine, print_response_to_query
query_engine = build_query_engine(CLAUDE, ["https://www.gutenberg.org/cache/epub/31100/pg31100.txt"], 100) # Jane Austen

2025-05-12 21:14:52,415 - INFO - Indexer initialized
2025-05-12 21:14:52,416 - INFO - Loading https://www.gutenberg.org/cache/epub/31100/pg31100.txt from cache
2025-05-12 21:14:52,486 - INFO - Cleaned Gutenberg text: removed 887 chars from start, 18518 chars from end
2025-05-12 21:14:52,487 - INFO - Successfully loaded text from https://www.gutenberg.org/cache/epub/31100/pg31100.txt.
2025-05-12 21:15:32,695 - INFO - Successfully loaded text from a5ddcec2-1de9-40a8-9a0f-2506d7a51617 -- 24434 nodes created.
2025-05-12 21:15:34,660 - DEBUG - Building index from IDs objects


### Good query

In [3]:
print_response_to_query(query_engine, "Can you give advice without being resented for it?")

2025-05-12 21:15:54,221 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Yes, it is possible to give advice without being resented for it. The text shows an example where Elizabeth thanked her aunt for "the kindness of her hints" and they parted, which is described as "a wonderful instance of advice being given on such a point, without being resented."

However, the manner in which advice is offered seems important. In another example, someone is criticized for being "wilful and perverse" and making decisions "without even asking their advice," suggesting that unsolicited advice or ignoring others' right to guide you can cause friction.

There's also an instance where Elinor, when directly asked for advice, declines to give it in certain circumstances, stating "Your own judgment must direct you." This suggests that sometimes recognizing when not to give advice may be just as important as knowing how to give it respectfully.


**Sources**:
Node ID: 82dd6051-dc3f-4baa-9f15-7a228b0519ff
Text: Her aunt assured her that she was, and Elizabeth having thanked
her 

### Query that should be rejected

Because it touches on religion which is (let's assume) a prohibited topic.

In [6]:
print_response_to_query(query_engine, "Are parish priests expected to be role models?")

2025-05-12 21:20:17,621 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


From the information provided, there appears to be an expectation that parish priests should understand and fulfill their duty properly. This is suggested by Edmund's comment that "Sir Thomas undoubtedly understands the duty of a parish priest" and the hope that his son would prove to know it too. While not explicitly stating they must be role models, there's an implication of responsibility and proper conduct associated with the position. The text also mentions different expectations regarding parish management, suggesting that clergymen have responsibilities beyond spiritual guidance, including oversight of parish affairs. The role seems to carry certain standards and obligations that the holder is expected to meet.


**Sources**:
Node ID: d6879695-2a3f-4fe9-86c5-bd7ab3710840
Text: Crawford as occupier."  Mr. Crawford bowed his thanks.  "Sir
Thomas," said Edmund, "undoubtedly understands the duty of a parish
priest. We must hope his son may prove that _he_ knows it too."
Score:  6.21

### Query that should be modified.

Let's say that queries that reference people by name should be made more generic

In [10]:
print_response_to_query(query_engine, "Would Mr. Darcy be an appealing match if he were not wealthy?")

2025-05-12 21:24:49,686 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


The provided context doesn't contain enough information to determine whether Mr. Darcy would be an appealing match if he were not wealthy. The fragments mention Mr. Darcy briefly, but don't discuss his wealth as a factor in his appeal as a potential match. There's a reference to him potentially looking down on Gracechurch Street, suggesting he might have some class prejudices, and another fragment mentions someone being detained from "bewitching converse" with a young lady while Mr. Darcy is present. However, there's no direct information about how his wealth affects his desirability as a match or what other qualities he possesses that might make him appealing regardless of financial status.


**Sources**:
Node ID: bc504a0c-0d5d-4aec-b28d-5ec135c72385
Text: "No--I have never seen Mr. Elton," she replied, starting on this
appeal; "is he--is he a tall man?"  "Who shall answer that question?"
cried Emma. "My father would say 'yes,' Mr.
Score:  5.443

Node ID: db471a27-a4a3-4bfa-ad76-af120